In [2]:
!pip install catboost
!pip install lightgbm
!pip install optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from datetime import datetime
import time 
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 1000)
import scipy.stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import optuna
from scipy import stats
from scipy.stats import boxcox
from scipy.special import boxcox1p
from sklearn import linear_model

In [22]:
orders = pd.read_csv('Orders.csv')
orders.isna().sum()

Row.ID                0
Order.ID              0
Order.Date            0
Ship.Date             0
Ship.Mode             0
Customer.ID           0
Customer.Name         0
Segment               0
Postal.Code       41296
City                  0
State                 0
Country               0
Region                0
Market                0
Product.ID            0
Category              0
Sub.Category          0
Product.Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping.Cost         0
Order.Priority        0
dtype: int64

In [19]:
orders['Profit'] = orders['Profit'].str.replace('$','')
orders['Profit'] = orders['Profit'].str.replace(',','')
orders['Profit'] = pd.to_numeric(orders['Profit'])

<ipython-input-19-db2e8bd710f2>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  orders['Profit'] = orders['Profit'].str.replace('$','')


In [20]:
orders['Sales'] = orders['Sales'].str.replace('$','')
orders['Sales'] = orders['Sales'].str.replace(',','')
orders['Sales'] = pd.to_numeric(orders['Sales'])

<ipython-input-20-bce5f72a4dfb>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  orders['Sales'] = orders['Sales'].str.replace('$','')


In [27]:
returns = pd.read_csv('Returns.csv')
returns.head()

,Returned,Order ID,Region
0,Yes,CA-2012-SA20830140-41210,Central US
1,Yes,IN-2012-PB19210127-41259,Eastern Asia
2,Yes,CA-2012-SC20095140-41174,Central US
3,Yes,IN-2015-JH158207-42140,Oceania
4,Yes,IN-2014-LC168857-41747,Oceania


In [28]:
returns['Order.ID'] = returns['Order ID']
returns.drop(columns='Order ID', inplace=True)
total = pd.merge(orders, returns, on=['Order.ID','Region'])

In [37]:
total['Profit'] = total['Profit'].str.replace('$','')
total['Profit'] = total['Profit'].str.replace(',','')
total['Profit'] = pd.to_numeric(total['Profit'])
total['Sales'] = total['Sales'].str.replace('$','')
total['Sales'] = total['Sales'].str.replace(',','')
orders['Sales'] = pd.to_numeric(total['Sales'])

<ipython-input-37-b6c38bb4110e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  total['Profit'] = total['Profit'].str.replace('$','')
<ipython-input-37-b6c38bb4110e>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  total['Sales'] = total['Sales'].str.replace('$','')


In [47]:
total.groupby('Returned').agg({'Profit': 'sum'})

,Profit
Returned,
Yes,60769.25


In [50]:
total['Order.Date'] = pd.to_datetime(total['Order.Date'])
total['Ship Date'] = pd.to_datetime(total['Ship.Date'])

In [52]:
total['Year'] = pd.DatetimeIndex(total['Order.Date']).year
total.groupby(['Year','Returned']).agg({'Profit': 'sum'})

In [59]:
total.groupby(['Year','Returned']).agg({'Profit': 'sum'})

,,Profit
Year,Returned,
2012,Yes,17396.86
2013,Yes,9077.77
2014,Yes,17180.12
2015,Yes,17114.50


In [67]:
frequent_returners = total.groupby(['Customer.ID','Returned']).agg({'Customer.ID': 'count'})
five_or_more = frequent_returners.loc[frequent_returners['Customer.ID'] > 5]
five_or_more.shape

(46, 1)

In [78]:
region_returners = total.groupby(['Region','Returned']).agg(count= ('Region', 'count'))
region_returners.sort_values('count')

,,count
Region,Returned,
Central Asia,Yes,9
Central Africa,Yes,17
Eastern Africa,Yes,18
Southern Africa,Yes,25
Eastern Europe,Yes,42
North Africa,Yes,51
Western Africa,Yes,60
Caribbean,Yes,69
Central US,Yes,71
